# Install required Python modules

*Note:* This step may take a while to run the first time.

In [1]:
# Install uv for simplified package management. (It is not necessary to understand this step.)
!curl -LsSf https://astral.sh/uv/install.sh | sh

downloading uv 0.8.4 x86_64-unknown-linux-gnu
no checksums to verify
installing to /opt/app-root/src/.local/bin
  uv
  uvx
everything's installed!


In [2]:
# Step 1: Install necessary libraries (run in a cell if needed)
!uv pip install -r requirements.txt

Using Python 3.11.11 environment at: /opt/app-root
Audited 2 packages in 4ms


# Validate LLM connectivity
Check that the LLM that is used by this lab is available.

In [3]:
import requests
import json

llm_status = "🟢 OK"

llm_uri = "http://llama3-2-3b-predictor.llama-serving.svc.cluster.local:8080/v1/completions"
model_name = "llama3-2-3b"

payload = {
    "model": model_name,
    "prompt": "What is the capital of Australia?\nA:",
    "max_tokens": 50,
    "temperature": 0.7,
    "stop": ["\n"]  # Prevent the LLM from generating extra questions as it responds.
}

headers = {"Content-Type": "application/json"}

try:
    response = requests.post(llm_uri, headers=headers, data=json.dumps(payload), timeout=10)
    if response.ok:
        result = response.json()
        print("✅ vLLM responded successfully:")
        print("Completion:", result["choices"][0]["text"].strip())
    else:
        print(f"⚠️ vLLM responded with status code {response.status_code}:")
        print(response.text)
        llm_status = "🔴 FAIL"
except requests.exceptions.RequestException as e:
    print(f"❌ Failed to reach vLLM at {llm_uri}: {e}")
    llm_status = "🔴 FAIL"

print(f"LLM status: {llm_status}")

print("\nIf all results are OK, return to your lab workbook for further instructions. If any have an error please inform your instructor.")

✅ vLLM responded successfully:
Completion: Canberra
LLM status: 🟢 OK

If all results are OK, return to your lab workbook for further instructions. If any have an error please inform your instructor.
